In [ ]:
!pip install pypianoroll

In [1]:
import numpy as np
import pypianoroll as ppr
import glob
import pickle
import multiprocessing

pool = multiprocessing.Pool(multiprocessing.cpu_count())

# Paths to the data
path_train = './train/*.mid'
path_valid = './validation/*.mid'
path_test = './test/*.mid'

path_iter = sorted(glob.glob(path_train))

In [2]:
def midi2pr(filename, chunk_size = 512):
    """
    This functions receives a filename, reads the associated midi file,
    extracts the piano roll and returns a list of arrays of size (chunk_size,72)
    """
    pr = ppr.parse(filename, beat_resolution = 8)
    pr = pr.tracks[0].pianoroll
    pr_list = [pr[i:i+chunk_size] for i in range(0, pr.shape[0], chunk_size)][:-1]
    square_list = []
    for pr in pr_list:
        square = np.empty((256,256))
        square[:128,:128] = pr[:128,:]
        square[:128,128:] = pr[128:256,:]
        square[128:,:128] = pr[256:384,:]
        square[128:,128:] = pr[384:,:]
        square_list.append(square)
    print("ok")
    return np.array(square_list)

In [3]:
from tqdm import notebook

#path_iter = iter(sorted(glob.glob(path_train)))
'''
tr_array = midi2pr(next(path_iter))

i = 0
for filename in notebook.tqdm(path_iter, total=926):
    tr_array = np.concatenate((tr_array, midi2pr(filename)))
    i+=1
    print(i)
'''

tr_array = np.array([pool.apply(midi2pr, filename) for filename in path_iter])

In [ ]:
path_iter = iter(sorted(glob.glob(path_valid)))

val_array = midi2pr(next(path_iter))

for filename in path_iter:
    val_array = np.concatenate((val_array, midi2pr(filename)))

In [ ]:
path_iter = iter(sorted(glob.glob(path_test)))

test_array = midi2pr(next(path_iter))

for filename in path_iter:
    test_array = np.concatenate((test_array, midi2pr(filename)))

In [ ]:
pickle.dump(tr_array, open("train_sq.pickle", 'wb'))
pickle.dump(val_array, open("valid_sq.pickle", 'wb'))
pickle.dump(test_array, open("test_sq.pickle", 'wb'))